# Export Large Dataset to Spark
The current competition data file size is about 30GB+. We can't use `pandas` because it will take a long time to even read it. You can switch to `dask` or `pyspark` for this kind of problem. In this notebook I will choose `pyspark` because I want to apply what have I learned about data engineering.

The real reason we use `pyspark` is because it runs operations using multiple machine while `pandas` only use single machine. `pyspark` can perform lazy operation so that we have no to wait every operations to be finished. If you try to read the data using `pandas` it will take a long long time to even finished the read operations, that isn't a good practice. Some may use `cudf` and that is quite a good idea, but we will stick to `pyspark`.

The downside is `pyspark` have less algorithms than `pandas`, it might restraining our flexibility.

## Install pyspark

In [ ]:
!pip install -q pyspark

## Import Libraries

In [ ]:
import os
from pprint import pprint

import pandas as pd
from pyspark.sql import SparkSession, types

## Spark Session
In order to use `pyspark` we need to create or get `spark` instance.

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

## Infer Data Types
When we read the data directly with `pyspark` it regards all the data types as string. While for `pandas`, it tries to infer what the data types by the value. We will use `pandas` for this task, but we can't afford to read all the data by whole. However, we only read the first n rows of the data and fetch all the data types.

In [ ]:
# Define data paths
test_path = "../input/amex-default-prediction/test_data.csv"
train_path = "../input/amex-default-prediction/train_data.csv"
label_path = "../input/amex-default-prediction/train_labels.csv"

In [ ]:
# Read data
train_df = pd.read_csv(train_path, nrows=100)
test_df = pd.read_csv(test_path, nrows=100)
label_df = pd.read_csv(label_path, nrows=100)

In [ ]:
# Get all data types

## Train types
train_types = train_df.dtypes
train_types_count = train_types.value_counts()

## Test types
test_types = test_df.dtypes
test_types_count = test_types.value_counts()

## Label types
label_types = label_df.dtypes
label_types_count = label_types.value_counts()

In [ ]:
def print_splits(*msg):
    for m in msg:
        print(m)
        print()

We get all the data types

In [ ]:
print_splits(train_types_count, test_types_count, label_types_count)

## Create Schemas
After we have retrieved the data types, we can create spark schema by creating `StructType` instances with every column as the argument. Every column is defined using `StructField` instance, it receives 3 arguments `colname`, `data types` and `nullable`.

In [ ]:
# Types mapper
types_map = {
    "object": types.StringType(),
    "float64": types.FloatType(),
    "int64": types.IntegerType(),
}

# Known dtypes
string_dtypes = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']
date_dtypes = ['S_2']

In [ ]:
def create_spark_schema(series):
    fields = []
    
    for index, value in series.items():
        if index in string_dtypes:
            field = types.StructField(index, types.StringType(), True)
            
        elif index in date_dtypes:
            field = types.StructField(index, types.DateType(), True)
        
        else:
            field = types.StructField(index, types_map.get(str(value)), True)
            
        fields.append(field)
    return types.StructType(fields)

In [ ]:
train_schema = create_spark_schema(train_types) 
test_schema = create_spark_schema(test_types)
label_schema = create_spark_schema(label_types)

## Read with Pyspark

In [ ]:
# Set header to True or else it will be included as row
train_psdf = spark.read.option("header", "true").csv(train_path, schema=train_schema)
test_psdf = spark.read.option("header", "true").csv(test_path, schema=test_schema)
label_psdf = spark.read.option("header", "true").csv(label_path, schema=label_schema)

In [ ]:
# Check schema
print_splits(train_psdf.schema[:3], test_psdf.schema[:3], label_psdf.schema[:3])

## Counts Data
The data is about 5 millions rows with 100+ columns

In [ ]:
train_psdf.count()

## Save as Parquet
We choose `.parquet` as the file extension because it uses less disk memory.

In [ ]:
(train_psdf.write
           .partitionBy("S_2")
           .parquet("train_amex"))

(test_psdf.write
          .partitionBy("S_2")
          .parquet("test_amex"))

label_psdf.write.parquet("label_amex")

## What to do next?
1. You can read the exported data with pyspark API it can be `pyspark.sql`, `pyspark.pandas`, or `pyspark.rdd`
2. Then perform preprocesing, you can see the related notebook from [Michal Slapek](https://www.kaggle.com/capslock): [here](https://www.kaggle.com/code/capslock/amex-export-to-parquet-with-apache-spark)

My coverage notebook about doing ML in pyspark for this competitions is still in progress. If anybody have done it I would like to know :D how it is done.

## Update

**Changes:**
- v.4 Added `.option("header", "true")` to not include header as row
- v.5 Update schema with known dtypes
- v.6 Failed Run
- v.7 Change the title to avoid misleading (the preprocessing notebook is still in progress)
- v.8 PartitionBy column "S_2"


Good luck for the competitions :D.